In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import mktime
import time
import re
from io import StringIO
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from selenium.webdriver.chrome.options import Options
options = Options()
# wait = WebDriverWait(driver, 20)
import os

from selenium.common.exceptions import StaleElementReferenceException



In [3]:
tickers = ['AALB.AS', 'ABN.AS', 'ACCEL.AS', 'AXS.AS', 'ADYEN.AS', 'AED.BR', 'AGN.AS', 'AD.AS', 'AFR.F', 'AJAX.AS', 'AKZA.AS', 'ALFEN.AS', 'ATCB.AS', 'ATC.AS', 'ALX.AS', 'AMG.AS', 'ACOMO.AS', 'AND.AS', 'AAA.AS', 'APAM.AS', 'ARCAD.AS', 'MT.AS', 'ASM.AS', 'ASML.AS', 'ASRNL.AS', 'ATRS.AS', 'AVTX.AS', 'BSGR.AS', 'BAMNB.AS', 'BFIT.AS', 'BESI.AS', 'BBED.AS', 'BEVER.AS', 'BOKA.AS', 'BGHL.AS', 'BGHS.L', 'BRILL.AS', 'BRNL.AS', 'CMCOM.AS', 'CCEP.AS', 'CRBN.AS', 'CLB.AS', 'CTAC.AS', 'DGB.AS', 'DPA.AS', 'DSM.AS', 'EAS2P.AS', 'ENVI.AS', 'ESP.AS', 'ECT.AS', 'ECMPA.AS', 'ENX.PA', 'FAGR.BR', 'FAST.AS', 'FLOW.AS', 'FFARM.AS', 'FUR.AS', 'GLPG.AS', 'GVNV.AS', 'HAL.AS', 'HEIJM.AS', 'HEIA.AS', 'HEIO.AS', 'HOLCO.AS', 'HDG.AS', 'HUNDP.AS', 'HYDRA.AS', 'ICT.AS', 'IEX.AS', 'IMCD.AS', 'INGA.AS', 'INTER.AS', 'JDEP.AS', 'TKWY.AS', 'KARD.AS', 'KENDR.AS', 'KDS.AS', 'KPN.AS', 'LVIDE.AS', 'BOLS.AS', 'MAREL.AS', 'NEDSE.AS', 'MORE.AS', 'NEDAP.AS', 'NRP.AS', 'NSE.AS', 'NEWAY.AS', 'NIBC.AS', 'NN.AS', 'NOVI.AS', 'NSI.AS', 'OCI.AS', 'ORANW.AS', 'ORDI.AS', 'PSH.AS', 'PHARM.AS', 'PHIA.AS', 'PORF.AS', 'PNL.AS', 'PRX.AS', 'RAND.AS', 'REINA.AS', 'REN.AS', 'RWI.AS', 'RET.BR', 'ROOD.AS', 'RDSA.AS', 'RDSB.AS', 'SGO.PA', 'SBMO.AS', 'SIFG.AS', 'LIGHT.AS', 'SLIGR.AS', 'SNOW.AS', 'STRN.AS', 'TFG.AS', 'TBIRD.AS', 'TIE.AS', 'TWEKA.AS', 'TOM2.AS', 'URW.AS', 'UNA.AS', 'VLK.AS', 'VALUE.AS', 'PREVA.AS', 'VASTN.AS', 'VEON.AS', 'VVY.AS', 'VTA.AS', 'VPK.AS', 'WDP.BR', 'WHA.AS', 'WKL.AS', 'YATRA.AS']


In [47]:
df2

Category,Ticker,Basic Average Shares
2019,AALB.AS,"110,580"


In [2]:
def get_data(ticker):
    url = 'https://finance.yahoo.com/quote/{0}/financials?p={0}'.format(ticker)
    
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    table_tag = soup.find('div', class_='D(tbrg)')

    table_tag = BeautifulSoup(str(table_tag), 'html.parser')
    
    rows = table_tag.find_all('div', attrs={'class':'D(tbr) fi-row Bgc($hoverBgColor):h'})
    df = pd.DataFrame()

    for row in rows:
        time = row.find_all('span', attrs={'class':'Va(m)'})

        cells = row.find_all('div', attrs={'class':['Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)'
        ,'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(tbc)']})

        try:
            d = {

            'Category' : re.search('>(.+?)<', str(time)).group(1),     
            '2019' : cells[1].text if len(cells) >= 2 else 0,
            '2018' : cells[2].text if len(cells) >= 3 else 0,
            '2017' : cells[3].text if len(cells) >= 4 else 0,    

            }

            df = df.append(d, ignore_index=True)  

        
        except KeyError:
    # #         print('Ticker {} not found'.format(ticker))
            pass
    df.set_index('Category',inplace=True)
    df = df.T
    df.reset_index(level=0, inplace=True)
    
    df['Ticker'] = ticker
    
    df = df[['index','Ticker','Basic Average Shares']]
    df = df.apply(lambda x:x.str.replace('-',''))
    df = df.apply(lambda x:x.str.replace(',',''))
#     df = df[['Ticker','Basic Average Shares']]
    cols=[i for i in df.columns if i not in ["Category",'Ticker']]
    for col in cols:
        df[col]=pd.to_numeric(df[col])

    df = df.fillna(0)
    df = df[df['Basic Average Shares']!=0]
#     max_year = df['index'].max()  
    df2 = df[df['index']==df['index'].max()  ]
    return df2

In [145]:
df.columns

Index(['index', 'Total Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expense', 'Operating Income',
       'Net Non Operating Interest Income Expense', 'Pretax Income',
       'Tax Provision', 'Net Income Common Stockholders',
       'Diluted NI Available to Com Stockholders', 'Basic EPS', 'Diluted EPS',
       'Basic Average Shares', 'Diluted Average Shares',
       'Total Operating Income as Reported', 'Total Expenses',
       'Net Income from Continuing &amp; Discontinued Operation',
       'Normalized Income', 'Interest Income', 'Interest Expense',
       'Net Interest Income', 'EBIT', 'EBITDA', 'Reconciled Cost of Revenue',
       'Reconciled Depreciation',
       'Net Income from Continuing Operation Net Minority Interest',
       'Total Unusual Items Excluding Goodwill', 'Total Unusual Items',
       'Normalized EBITDA', 'Tax Rate for Calcs',
       'Tax Effect of Unusual Items', 'Ticker'],
      dtype='object', name='Category')

In [127]:
df['index'].max()  

'2019'

In [3]:
df = get_data('ADYEN.AS')
df

Category,index,Ticker,Basic Average Shares
1,2018,ADYEN.AS,29487.0


In [5]:
def download_tickers(tickets):

    df = pd.DataFrame()
    for ticker in tickers:
    #     df = get_data(ticker,driver)
        print(ticker)
        try:
            df = pd.concat([df,get_data(ticker)])
        except KeyError:
            pass
    return df        

In [159]:
df.to_csv('OutstandingShares.csv')

In [96]:
df = df.apply(lambda x:x.replace('-',0).replace(',',''))

In [121]:
# df.groupby(['Ticker'])['Basic Average Shares'].max()

df = df.apply(lambda x:x.str.replace(',',''))
#     try: 
#         df['time'] = pd.to_datetime(df['time']).dt.strftime('%Y%m%d')
cols=[i for i in df.columns if i not in ['Ticker']]
for col in cols:
    df[col]=pd.to_numeric(df[col])
    
df.fillna(0)
# df = df.sort_values('time', ascending = True).reset_index(drop=True)

AttributeError: Can only use .str accessor with string values!

In [111]:
df = df.fillna(0)

In [117]:
df = df[df['Basic Average Shares']!=0]

In [118]:
df[]

Category,Ticker,Basic Average Shares
2017,AALB.AS,110580.0
2018,AALB.AS,110580.0
2019,AALB.AS,110580.0
2017,ABN.AS,940000.0
2018,ABN.AS,940000.0
...,...,...
2019,WHA.AS,40252.0
2017,WKL.AS,285100.0
2018,WKL.AS,276700.0
2017,YATRA.AS,3047.0


In [106]:
df.groupby(['Ticker'])['Basic Average Shares']

Ticker
AALB.AS      110580.0
ABN.AS       940000.0
ACCEL.AS      27020.0
ACOMO.AS      24650.0
AD.AS       1251000.0
              ...    
VVY.AS         8372.0
WDP.BR       164047.0
WHA.AS        40252.0
WKL.AS       285100.0
YATRA.AS       3047.0
Name: Basic Average Shares, Length: 130, dtype: float64

In [ ]:
# soup = BeautifulSoup(html, 'html.parser')
# table_tag = soup.find('div', class_='D(tbrg)')

# table_tag = BeautifulSoup(str(table_tag), 'html.parser')

# rows = table_tag.find_all('div', attrs={'class':'D(tbr) fi-row Bgc($hoverBgColor):h'})

# df = pd.DataFrame()